# Comparison of GRCWA NLOPT Optimisation with NIDN Siren and NIDN Voxel.

In the following we compare an optimisation using GRCWA with NIDN using a Siren network and NIDN when directly encoding permittivity as a voxel grid.

The test case is the design of a material that is maximally reflective.

1. GRCWA - Optimisation as described here https://github.com/weiliangjinca/grcwa/blob/master/example/ex3.py
2. NIDN Siren - As in other notebooks
3. NIDN Voxel - Weights are directly permittivities of the material.

Let's start with imports

In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../..")

import nidn
import nlopt
import grcwa
grcwa.set_backend('autograd')
import torch
import numpy as npf
import autograd.numpy as np
from autograd import grad

# Start with the default config
cfg = nidn.load_default_cfg()

Let's define grid specifics etc. both for GRCWA and NIDN

In [ ]:
# Set grid specifics
# Change to 32 to generate grid plot with more than 1 element displayed
cfg.Nx = 1
cfg.Ny = 1
cfg.N_layers = 5
cfg.N_freq = 20
cfg.TRCWA_L_grid = [[0.1,0.0],[0.0,0.1]]
cfg.TRCWA_NG = 11
cfg.PER_LAYER_THICKNESS = [1.0]
cfg.freq_distribution = "log"

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = 2e-6
cfg.physical_wavelength_range[1] = 1e-5

# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

# Allowed range of epsilon values
cfg.real_min_eps = 0.01
cfg.real_max_eps = 20.0
cfg.imag_min_eps = 0.0
cfg.imag_max_eps = 0.0 # We only allow real permittivity values for easy comparison with GRCWA using NLOPT

In [ ]:
# Truncation order (actual number might be smaller)
nG = 11
# lattice constants
L1 = [0.1,0]
L2 = [0,0.1]
# frequency and angles

theta = 0.
phi = 0.

# now consider 3 layers: vacuum + patterned + vacuum
ep0 = 1. # dielectric for layer 1 (uniform)
epN = 1.  # dielectric for layer N (uniform)

thick0 = 1. # thickness for vacuum layer 1
thickN = 1.

# planewave excitation
planewave={'p_amp':0,'s_amp':1,'p_phase':0,'s_phase':0}


# Function to compute R,T with GRCWA
def fun_reflection(x,Qabs,freq):
    freqcmp = freq*(1+1j/2/Qabs)
    ######### setting up RCWA
    obj = grcwa.obj(nG,L1,L2,freqcmp,theta,phi,verbose=0)
    # input layer information
    obj.Add_LayerUniform(thick0,ep0)
    obj.Add_LayerUniform(thick0,x[:,:,0][0])
    obj.Add_LayerUniform(thick0,x[:,:,1][0])
    obj.Add_LayerUniform(thick0,x[:,:,2][0])
    obj.Add_LayerUniform(thick0,x[:,:,3][0])
    obj.Add_LayerUniform(thick0,x[:,:,4][0])
    obj.Add_LayerUniform(thickN,epN)
    obj.Init_Setup()

    obj.MakeExcitationPlanewave(planewave['p_amp'],planewave['p_phase'],planewave['s_amp'],planewave['s_phase'],order = 0)    
    R,T= obj.RT_Solve(normalize=1)
    return R,T

# Function to compute summed reflection, we aim to maximise this.
def fun_total_reflection(x,Qabs=1e5):
    total_R = 0
    x = x.reshape(1,1,cfg.N_layers,cfg.N_freq)
    for idx,freq in enumerate(cfg.target_frequencies):
        R,T = fun_reflection(x[:,:,:,idx],1e16,freq)
        total_R += R
    return total_R


### Define optimisation problem for GRCWA with NLOPT

In [ ]:

ctrl = 0
fun = lambda x: fun_total_reflection(x)
grad_fun = grad(fun)


In [ ]:
def fun_nlopt(x,gradn):
    global ctrl
    gradn[:] = grad_fun(x)
    y = fun(x)
    
    print('Step = ',ctrl,', R = ',y)
    ctrl += 1
    return fun(x)

# set up NLOPT
ndof = cfg.N_layers*cfg.N_freq*cfg.Nx*cfg.Ny

# Sample randomly in 0 to 20 as for neural net range
init = cfg.real_min_eps + (cfg.real_max_eps-cfg.real_min_eps)*np.random.random(ndof)
lb=np.ones(ndof,dtype=float)*0.01
ub=np.ones(ndof,dtype=float)*20.0

opt = nlopt.opt(nlopt.LD_MMA, ndof)
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)

opt.set_xtol_rel(1e-16)
opt.set_maxeval(2000)

opt.set_max_objective(fun_nlopt)
x = opt.optimize(init)

### Plot achieved spectrum

In [ ]:
x = x.reshape(1,1,cfg.N_layers,cfg.N_freq)
Rs,Ts = [],[]
for idx,freq in enumerate(cfg.target_frequencies):
    R,T = fun_reflection(x[:,:,:,idx],1e16,freq)
    Rs.append(R)
    Ts.append(T)

nidn.plot_spectrum(cfg,Rs,Ts)

### Set up VoxelGrid optimisation with NIDN

In [ ]:
cfg.pop("model",None); # Forget the old model

cfg.model_type = "voxel"
cfg.learning_rate = 5e-2 # LR can be much higher here as we directly optimise the permittivity

# Define reflection only spectrum
cfg.target_reflectance_spectrum = [1]*cfg.N_freq
cfg.target_transmittance_spectrum = [0]*cfg.N_freq

cfg.type = "regression" # Choose type as described above
cfg.iterations = 2000 # Set number of training iterations (that is forward model evaluations) to perform
nidn.set_log_level("INFO")

In [ ]:
nidn.run_training(cfg);

### Plot and save results

In [ ]:
nidn.plot_spectra(cfg)
cfg.name = "voxel_run"
nidn.save_run(cfg)

### Now do the same with NIDN Siren

In [ ]:
cfg.pop("model",None); # Forget the old model

cfg.model_type = "siren"
cfg.learning_rate = 8e-5

cfg.siren_omega = 1.

cfg.type = "regression" # Choose type as described above
cfg.iterations = 2000 # Set number of training iterations (that is forward model evaluations) to perform
nidn.set_log_level("INFO")

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_spectra(cfg)
cfg.name = "siren_run"
nidn.save_run(cfg)